# Loading libaries

In [1]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from scipy import stats
import matplotlib.pyplot as plt
from copy import deepcopy
from plotly.subplots import make_subplots

# Simple Sine

## Loading Data

In [4]:
DIR = 'Results/Simple_Sine/Catch22/'

In [5]:
df_real_1 = pd.read_csv(DIR+'features_df_Sine_signal_1_real_21.csv', index_col=0).add_prefix('real_1_')
df_real_2 = pd.read_csv(DIR+'features_df_Sine_signal_2_real_21.csv', index_col=0).add_prefix('real_2_')

df_generated_without_CD_1 = pd.read_csv(DIR+'Without_CD/features_df_signal_1_generated_data_frame_21.csv', index_col=0).add_prefix('generated_without_CD_1_')
df_generated_without_CD_2 = pd.read_csv(DIR+'Without_CD/features_df_signal_2_generated_data_frame_21.csv', index_col=0).add_prefix('generated_without_CD_2_')

df_generated_with_CD_1 = pd.read_csv(DIR+'With_CD/features_df_signal_1_generated_data_frame_21.csv', index_col=0).add_prefix('generated_with_CD_1_')
df_generated_with_CD_2 = pd.read_csv(DIR+'With_CD/features_df_signal_2_generated_data_frame_21.csv', index_col=0).add_prefix('generated_with_CD_2_')

df_real_1

,real_1_DN_HistogramMode_5,real_1_DN_HistogramMode_10,real_1_CO_f1ecac,real_1_CO_FirstMin_ac,real_1_CO_HistogramAMI_even_2_5,real_1_CO_trev_1_num,real_1_MD_hrv_classic_pnn40,real_1_SB_BinaryStats_mean_longstretch1,real_1_SB_TransitionMatrix_3ac_sumdiagcov,real_1_PD_PeriodicityWang_th0_01,...,real_1_DN_OutlierInclude_p_001_mdrmd,real_1_DN_OutlierInclude_n_001_mdrmd,real_1_SP_Summaries_welch_rect_area_5_1,real_1_SB_BinaryStats_diff_longstretch0,real_1_SB_MotifThree_quantile_hh,real_1_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,real_1_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,real_1_SP_Summaries_welch_rect_centroid,real_1_FC_LocalSimple_mean3_stderr,real_1_ID
0,-0.835185,-1.032904,39.0,2.0,0.714902,-0.000612,0.904881,96.0,0.017778,192.0,...,0.046250,0.27000,0.956345,5.0,1.557714,0.829787,0.680851,0.03068,0.269528,1.0
1,1.429570,1.247639,39.0,100.0,0.896222,-0.000276,0.877347,98.0,0.017778,197.0,...,0.031250,0.26250,0.976847,5.0,1.476074,0.829787,0.680851,0.03068,0.198519,2.0
2,1.372831,1.199161,39.0,100.0,1.047874,-0.000164,0.857322,98.0,0.017778,197.0,...,0.037500,0.26000,0.984703,5.0,1.419796,0.829787,0.680851,0.03068,0.163329,2.0
3,-0.841501,-1.040435,39.0,2.0,0.701817,-0.000645,0.907384,96.0,0.017778,192.0,...,0.047500,0.26875,0.954449,5.0,1.557714,0.829787,0.680851,0.03068,0.275173,1.0
4,1.380746,-0.017821,39.0,100.0,1.005855,-0.000177,0.859825,98.0,0.017778,197.0,...,0.017500,0.26000,0.983721,5.0,1.419796,0.829787,0.680851,0.03068,0.168131,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.420302,1.239722,39.0,100.0,0.926884,-0.000255,0.874844,98.0,0.017778,197.0,...,0.037500,0.25500,0.978262,5.0,1.476074,0.829787,0.680851,0.03068,0.192657,2.0
996,-0.845074,-1.044695,38.0,2.0,0.695717,-0.000665,0.908636,96.0,0.017778,192.0,...,0.047500,0.27750,0.953355,5.0,1.567879,0.829787,0.680851,0.03068,0.278375,1.0
997,1.438361,1.255148,39.0,100.0,0.873401,-0.000297,0.878598,98.0,0.017778,197.0,...,0.040000,0.26500,0.975456,5.0,1.476074,0.829787,0.680851,0.03068,0.204117,1.0
998,1.364368,1.191927,39.0,100.0,1.062280,-0.000150,0.853567,98.0,0.017778,197.0,...,0.030000,0.26000,0.985714,5.0,1.399466,0.829787,0.680851,0.03068,0.158235,2.0


## Correlation Analysis

### Computing Correlation

In [6]:
real_corr = pd.concat([df_real_1.drop(columns=["real_1_ID"]), df_real_2.drop(columns=["real_2_ID"])], axis=1).corr()
real_corr = real_corr[list(df_real_2.columns)[:-1]].loc[list(df_real_1.columns)[:-1]]

generated_without_CD = pd.concat([df_generated_without_CD_1, df_generated_without_CD_2], axis=1).corr()
generated_without_CD = generated_without_CD[list(df_generated_without_CD_2.columns)].loc[list(df_generated_without_CD_1.columns)]

generated_with_CD = pd.concat([df_generated_with_CD_1, df_generated_with_CD_2], axis=1).corr()
generated_with_CD = generated_with_CD[list(df_generated_with_CD_2.columns)].loc[list(df_generated_with_CD_1.columns)]

indices_real = real_corr.columns[real_corr.isna().all()].tolist()
indices_generated_without_CD = generated_without_CD.columns[generated_without_CD.isna().all()].tolist()
indices_generated_with_CD = generated_with_CD.columns[generated_with_CD.isna().all()].tolist()

indices_total = indices_real + indices_generated_without_CD + indices_generated_with_CD

features = list(set(['SB_BinaryStats_diff_longstretch0', 'FC_LocalSimple_mean1_tauresrat', 'SP_Summaries_welch_rect_centroid', 'SB_TransitionMatrix_3ac_sumdiagcov', 'PD_PeriodicityWang_th0_01', 'FC_LocalSimple_mean1_tauresrat', 'SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'IN_AutoMutualInfoStats_40_gaussian_fmmi', 'SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'FC_LocalSimple_mean1_tauresrat', 'SB_TransitionMatrix_3ac_sumdiagcov']))

real_features_2 = ['real_2_'+f for f in features]
generated_without_CD_features_2 = ['generated_without_CD_2_'+f for f in features]
generated_with_CD_features_2 = ['generated_with_CD_2_'+f for f in features]

real_features_1 = ['real_1_'+f for f in features]
generated_without_CD_features_1 = ['generated_without_CD_1_'+f for f in features]
generated_with_CD_features_1 = ['generated_with_CD_1_'+f for f in features]

real_not_nan = real_corr.drop(real_features_2, axis=1)
generated_with_CD_not_nan = generated_with_CD.drop(generated_with_CD_features_2, axis=1)
generated_without_CD_not_nan = generated_without_CD.drop(generated_without_CD_features_2, axis=1)

real_not_nan = real_not_nan.drop(real_features_1, axis=0)
generated_with_CD_not_nan = generated_with_CD_not_nan.drop(generated_with_CD_features_1, axis=0)
generated_without_CD_not_nan = generated_without_CD_not_nan.drop(generated_without_CD_features_1, axis=0)

In [7]:
real_not_nan

,real_2_DN_HistogramMode_5,real_2_DN_HistogramMode_10,real_2_CO_f1ecac,real_2_CO_FirstMin_ac,real_2_CO_HistogramAMI_even_2_5,real_2_CO_trev_1_num,real_2_MD_hrv_classic_pnn40,real_2_SB_BinaryStats_mean_longstretch1,real_2_CO_Embed2_Dist_tau_d_expfit_meandiff,real_2_DN_OutlierInclude_p_001_mdrmd,real_2_DN_OutlierInclude_n_001_mdrmd,real_2_SP_Summaries_welch_rect_area_5_1,real_2_SB_MotifThree_quantile_hh,real_2_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,real_2_FC_LocalSimple_mean3_stderr
real_1_DN_HistogramMode_5,-0.830074,-0.134315,0.882545,-0.161514,0.913791,-0.701503,-0.892184,0.893731,0.901884,-0.701708,0.636069,0.792016,-0.826374,-0.582629,-0.844235
real_1_DN_HistogramMode_10,-0.580417,-0.493692,0.384342,-0.413392,0.564691,-0.476976,-0.532681,0.571779,0.523673,-0.325007,0.255372,0.527420,-0.557276,-0.681699,-0.549044
real_1_CO_f1ecac,-0.538498,-0.426538,0.445211,-0.209762,0.685198,-0.835942,-0.623100,0.567759,0.641707,-0.336883,-0.071063,0.810689,-0.773038,-0.808085,-0.773753
real_1_CO_FirstMin_ac,-0.842105,-0.489429,0.623988,-0.237939,0.846463,-0.763141,-0.798015,0.827957,0.802745,-0.516843,0.368715,0.817725,-0.844382,-0.806727,-0.837609
real_1_CO_HistogramAMI_even_2_5,-0.861923,-0.198754,0.883650,-0.139940,0.993394,-0.868100,-0.989212,0.949778,0.996747,-0.798197,0.537983,0.938081,-0.962921,-0.765799,-0.972713
real_1_CO_trev_1_num,-0.760631,-0.326313,0.720126,0.059799,0.920135,-0.996908,-0.858899,0.811664,0.886154,-0.576986,0.195983,0.995479,-0.974659,-0.862409,-0.976073
real_1_MD_hrv_classic_pnn40,0.848955,0.200140,-0.852922,0.152144,-0.985940,0.875502,0.988746,-0.933257,-0.993499,0.801912,-0.515900,-0.941851,0.962168,0.774487,0.973583
real_1_SB_BinaryStats_mean_longstretch1,-0.602459,-0.362870,0.518756,0.218395,0.763127,-0.941097,-0.677904,0.623538,0.714649,-0.393280,-0.038152,0.903477,-0.864750,-0.831291,-0.857697
real_1_CO_Embed2_Dist_tau_d_expfit_meandiff,-0.848911,-0.143855,0.887136,-0.144208,0.983353,-0.837818,-0.995292,0.943006,0.999277,-0.836754,0.582400,0.914923,-0.944809,-0.724107,-0.955848
real_1_DN_OutlierInclude_p_001_mdrmd,0.369884,-0.090223,-0.391234,0.069534,-0.480688,0.392713,0.552987,-0.384894,-0.537536,0.581178,-0.414634,-0.439549,0.462549,0.330419,0.469268


### Plotting Correlation

In [8]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Real Data',
                                    'Without CD', 'With CD'))

fig.add_trace(
    px.imshow(real_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=1
)
# fig.update_layout(
#     yaxis_title = 'Channel 2',
#     xaxis_title = 'Channel 1',
#     )
fig.add_trace(
    px.imshow(generated_without_CD_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=2
)

fig.add_trace(
    px.imshow(generated_with_CD_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=3
)
fig.update_xaxes(showticklabels=False) 
fig.update_yaxes(showticklabels=False) 
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 1)
fig.update_yaxes(title_text="Feature Channel 2", row = 1, col = 1)
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 2)
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 3)

fig.update_layout(height=240, width=800, margin = {'l':0,'r':0,'t':23,'b':0},)
fig.show()

In [9]:
fig.write_image("Simple_Sine_correlation.pdf")

### Spearman's rho

In [10]:
SPR_without_CD_1 = stats.spearmanr(np.array(real_not_nan).flatten(), np.array(generated_without_CD_not_nan).flatten())
SPR_with_CD_1 = stats.spearmanr(np.array(real_not_nan).flatten(), np.array(generated_with_CD_not_nan).flatten())

print("SPR for Signal 1 without CD: ", SPR_without_CD_1)
print("SPR for Signal 1 with CD: ", SPR_with_CD_1)

SPR for Signal 1 without CD:  SpearmanrResult(correlation=-0.7614485882848716, pvalue=7.230463684131775e-44)
SPR for Signal 1 with CD:  SpearmanrResult(correlation=0.8475769068689424, pvalue=2.720888255368768e-63)


### Kendall's Tau

In [11]:
KT_without_CD_1 = stats.kendalltau(np.array(real_not_nan).flatten(), np.array(generated_without_CD_not_nan).flatten())
KT_with_CD_1 = stats.kendalltau(np.array(real_not_nan).flatten(), np.array(generated_with_CD_not_nan).flatten())

print("KT for Signal 1 without CD: : ", KT_without_CD_1)
print("KT for Signal 1 with CD: : ", KT_with_CD_1)

KT for Signal 1 without CD: :  KendalltauResult(correlation=-0.5694444444444444, pvalue=4.971821795512218e-37)
KT for Signal 1 with CD: :  KendalltauResult(correlation=0.7003174603174603, pvalue=4.174214921324681e-55)


### Frobenius Norm

In [12]:
FBNorm_without_CD_1 = np.linalg.norm(np.array(real_not_nan) - np.array(generated_without_CD_not_nan))
FBNorm_with_CD_1 = np.linalg.norm(np.array(real_not_nan )- np.array(generated_with_CD_not_nan))

print("Frobenius Signal for Real 1 without CD: ", FBNorm_without_CD_1)
print("Frobenius Signal for Real 1 with CD: ", FBNorm_with_CD_1)

Frobenius Signal for Real 1 without CD:  11.294563762242682
Frobenius Signal for Real 1 with CD:  5.6663676360746855


### MAE

In [13]:
MSE_without_CD_1 = np.mean(np.abs(np.array(real_not_nan) - np.array(generated_without_CD_not_nan)))
MSE_with_CD_1 = np.mean(np.abs(np.array(real_not_nan) - np.array(generated_with_CD_not_nan)))

print("MSE of Real Data 1 and Generated Data without CD 1: ", MSE_without_CD_1)
print("MSE of Real Data 1 and Generated Data with CD 1: ", MSE_with_CD_1)

MSE of Real Data 1 and Generated Data without CD 1:  0.6711513439057466
MSE of Real Data 1 and Generated Data with CD 1:  0.29766086653872936
